In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from IPython.display import display, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8), 'figure.dpi': 150})
np.set_printoptions(suppress=True, linewidth=120, precision=4)
warnings.filterwarnings('ignore', category=FutureWarning)

# --- Utility Functions ---
def note(msg): display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title): print(f"\n{80*'='}\n| {title.upper()} |\n{80*'='}")

note("Environment initialized. Using networkx for graph analysis.")

# Chapter 10.3: An Introduction to Network Economics

---

### Table of Contents
1.  [**The Economy as a Network: Concepts and Measures**](#concepts)
    - [Representing Networks: Adjacency Matrices](#adjacency)
    - [Canonical Network Models: ER, WS, and BA](#canonical-models)
    - [Measuring Node Importance: Centrality](#centrality)
2.  [**Application 1: Social Networks**](#social-networks)
    - [Social Learning: The DeGroot Model](#degroot)
    - [Information Diffusion and Threshold Models](#diffusion)
3.  [**Application 2: Financial Networks and Systemic Risk**](#financial-networks)
    - [Modeling Contagion](#contagion)
4.  [**Application 3: Production Networks and Aggregate Fluctuations**](#production-networks)
    - [The Acemoglu et al. (2012) Framework](#acemoglu)
5.  [**Summary and Key Takeaways**](#summary)
6.  [**Exercises**](#exercises)

<a id='concepts'></a>
## 1. The Economy as a Network: Concepts and Measures

This chapter introduces **Network Economics**, a field that analyzes how the structure of connections between economic agents shapes their behavior and drives aggregate outcomes. Classical economic models often assume agents interact anonymously through a central market mechanism. Network economics relaxes this assumption, recognizing that in the real world, **connections matter**. Who you know determines your job opportunities; who you trade with defines your supply chain; and who you learn from shapes your beliefs.

A network (or **graph**) is a collection of **nodes** (or vertices) and **edges** (or links) that connect pairs of nodes. This simple structure has a long intellectual history, dating back to Leonhard Euler's 1736 solution to the famous **Seven Bridges of Königsberg** problem, which is considered the first paper in graph theory. In economics, this framework is incredibly versatile:
*   **Nodes:** Can represent individuals, firms, banks, or countries.
*   **Edges:** Can represent friendships, trade relationships, loans, or supply links. Edges can be **undirected** (symmetric), **directed** (asymmetric), or **weighted** (representing connection strength).

<a id='adjacency'></a>
### Representing Networks: Adjacency Matrices

While visualizations are intuitive, the computational workhorse of network analysis is the **adjacency matrix**, $\mathbf{A}$. For a network with $N$ nodes, this is an $N \times N$ matrix where the entry $A_{ij}$ is 1 if there is an edge from node $i$ to node $j$, and 0 otherwise. For weighted networks, $A_{ij}$ can store the weight of the edge.

The adjacency matrix is more than a storage device; its mathematical properties reveal deep truths about the network. For instance, the entry $(i, j)$ of the matrix $\mathbf{A}^k$ ($\mathbf{A}$ raised to the power of $k$) gives the number of distinct paths of length $k$ from node $i$ to node $j$. The `networkx` library is the standard Python tool for creating and analyzing these network structures.

<a id='canonical-models'></a>
### Canonical Network Models

Different real-world processes give rise to networks with vastly different structures. We can generate and visualize three canonical models to understand their distinct topologies. Understanding these archetypes helps us think about the mechanisms that might generate the networks we observe in the economy:
- **Erdos-Renyi (ER) Random Graph:** Any two nodes are connected with a fixed probability `p`. This is a model of purely random interactions and serves as a crucial theoretical benchmark. Its degree distribution is Poisson-like.
- **Watts-Strogatz (WS) Small-World Graph:** Starts with a regular ring lattice and rewires some edges. This captures two key features of real social networks: high clustering (friends of friends are also friends) and short average path lengths ("six degrees of separation").
- **Barabasi-Albert (BA) Scale-Free Graph:** New nodes attach preferentially to existing nodes with high degree. This "preferential attachment" or "rich get richer" mechanism generates a power-law degree distribution with highly-connected "hubs." This topology is common in trade, citation, and financial networks.

In [ ]:
sec("Interactive Canonical Network Models")
import ipywidgets as widgets

def plot_network_and_distribution(G, ax_network, ax_dist, title):
    pos = nx.spring_layout(G, seed=42, k=0.7)
    nx.draw(G, pos, ax=ax_network, with_labels=False, node_color='skyblue', node_size=60, width=0.6, edge_color='gray', alpha=0.8)
    ax_network.set_title(title, fontsize=14)
    degrees = [G.degree(n) for n in G.nodes()]
    sns.histplot(degrees, ax=ax_dist, bins=max(15, len(set(degrees))//2), kde=False, color='steelblue')
    ax_dist.set_xlabel('Degree'); ax_dist.set_ylabel('Frequency'); ax_dist.grid(True)
    ax_dist.set_title('Degree Distribution', fontsize=12)

def interactive_network_plot(model_type='Erdos-Renyi', p=0.05, k=4, m=2):
    N_nodes = 150
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7), gridspec_kw={'width_ratios': [3, 1]})
    if model_type == 'Erdos-Renyi':
        G = nx.erdos_renyi_graph(n=N_nodes, p=p)
        title = f'Erdos-Renyi (p={p:.2f})'
    elif model_type == 'Watts-Strogatz':
        G = nx.watts_strogatz_graph(n=N_nodes, k=k, p=p)
        title = f'Watts-Strogatz (k={k}, p_rewire={p:.2f})'
    else: # Barabasi-Albert
        G = nx.barabasi_albert_graph(n=N_nodes, m=m)
        title = f'Barabasi-Albert (m={m})'
        ax2.set_xscale('log'); ax2.set_yscale('log')
    
    plot_network_and_distribution(G, ax1, ax2, title)
    plt.tight_layout(); plt.show()

note("Use the dropdown and sliders to explore how the parameters of each canonical model shape the resulting network topology and degree distribution.")
widgets.interact(interactive_network_plot, 
                 model_type=widgets.Dropdown(options=['Erdos-Renyi', 'Watts-Strogatz', 'Barabasi-Albert'], value='Erdos-Renyi'),
                 p=widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.05, description='p / p_rewire'),
                 k=widgets.IntSlider(min=2, max=10, step=2, value=4, description='k (WS)'),
                 m=widgets.IntSlider(min=1, max=5, step=1, value=2, description='m (BA)'));

<a id='centrality'></a>
### Measuring Node Importance: Centrality

A key question in network analysis is identifying the most "important" or "central" nodes. The definition of importance depends on the economic context, and different centrality measures capture different aspects of it.

*   **Degree Centrality:** The fraction of nodes a given node is connected to. It is a simple measure of local popularity. For node $i$, $C_D(i) = \frac{\text{deg}(i)}{N-1}$.
*   **Betweenness Centrality:** The fraction of shortest paths between all pairs of nodes that pass through a given node. This measures a node's importance as a **broker** or **gatekeeper**. For node $i$, $C_B(i) = \sum_{j \neq k \neq i} \frac{\sigma_{jk}(i)}{\sigma_{jk}}$, where $\sigma_{jk}$ is the number of shortest paths between $j$ and $k$.
*   **Eigenvector Centrality:** Measures a node's influence by proposing that connections to other high-scoring nodes contribute more to a node's score than connections to low-scoring nodes. It is the entry in the eigenvector corresponding to the largest eigenvalue of the adjacency matrix $\mathbf{A}$. It solves the equation $\lambda C_E = \mathbf{A} C_E$.
*   **PageRank:** A variant of Eigenvector Centrality, famous for its use in Google's original search algorithm. It can be interpreted as the probability that a "random surfer" clicking on links will end up at a particular node.

![Centrality Measures Diagram](../images/png/centrality_measures_diagram.png)

In [ ]:
sec("Visualizing Centrality Measures")

def plot_centrality(G, centrality_func, ax, title):
    """Helper to plot a network with nodes colored and sized by centrality."""
    centrality = centrality_func(G)
    max_val = max(centrality.values()) if centrality else 1.0
    norm_centrality = {k: v / max_val for k, v in centrality.items()}
    node_color = list(norm_centrality.values())
    node_size = [v * 2000 + 100 for v in norm_centrality.values()]
    pos = nx.spring_layout(G, seed=42)
    
    nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_color=node_color, 
                                   node_size=node_size, cmap=plt.cm.viridis, alpha=0.9)
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='gray', alpha=0.5)
    plt.colorbar(nodes, ax=ax, shrink=0.8, label='Normalized Centrality')
    ax.set_title(title, fontsize=16)
    ax.set_axis_off()

# Use a classic network example: Zachary's Karate Club, a well-studied social network of a 1970s university karate club.
G_karate = nx.karate_club_graph()

fig, axes = plt.subplots(2, 2, figsize=(16, 14))
fig.suptitle("Figure 2: Centrality Measures on Zachary's Karate Club Network", fontsize=22, y=1.0)

plot_centrality(G_karate, nx.degree_centrality, axes[0, 0], 'a) Degree Centrality')
plot_centrality(G_karate, nx.betweenness_centrality, axes[0, 1], 'b) Betweenness Centrality')
plot_centrality(G_karate, nx.eigenvector_centrality, axes[1, 0], 'c) Eigenvector Centrality')
plot_centrality(G_karate, nx.pagerank, axes[1, 1], 'd) PageRank')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

note("The 'most important' nodes (brighter colors, larger size) differ depending on the measure. The club's administrator (node 0) and instructor (node 33) are consistently central, but their relative importance varies. Betweenness centrality, for example, highlights nodes that bridge the two factions of the club that eventually formed after a dispute.")

<a id='social-networks'></a>
## 2. Application 1: Social Networks

Networks are crucial for understanding how opinions, beliefs, and behaviors spread and evolve through social interactions.

<a id='degroot'></a>
### Social Learning: The DeGroot Model
The **DeGroot model** is a foundational and simple model of social learning. Agents in a network start with an initial belief (e.g., about the quality of a product or the state of the economy) and repeatedly update it by taking a weighted average of their neighbors' beliefs.

The update rule for the vector of beliefs $\mathbf{b}$ at time $t+1$ is a simple matrix multiplication:
$$ \mathbf{b}_{t+1} = \mathbf{T} \mathbf{b}_{t} $$
Where $\mathbf{T}$ is a **stochastic matrix** (rows sum to 1) representing the network's influence structure. A common specification is that an agent places equal weight on all their neighbors (including themselves). The long-run beliefs are determined by the properties of this matrix, specifically its stationary distribution, which corresponds to the eigenvector associated with the eigenvalue of 1.

In [ ]:
sec("Simulating the DeGroot Model of Social Learning")

def degroot_simulation(G, n_steps=20):
    n = len(G)
    # Create the influence matrix T. Add self-loops for agents to weight their own opinion.
    H = G.copy()
    H.add_edges_from([(i, i) for i in H.nodes()])
    T = nx.to_numpy_array(H)
    # Normalize rows to be stochastic (each row sums to 1)
    T = T / T.sum(axis=1, keepdims=True)
    
    # Initial beliefs are random numbers between 0 and 1
    rng = np.random.default_rng(123)
    b0 = rng.random((n, 1))
    
    # Simulate the evolution of beliefs by repeatedly applying the matrix
    belief_history = [b0]
    b_t = b0
    for _ in range(n_steps):
        b_t = T @ b_t
        belief_history.append(b_t)
    return np.array(belief_history).squeeze()

# Use a network that is strongly connected
G_learn = nx.barabasi_albert_graph(n=50, m=2, seed=123)
history = degroot_simulation(G_learn)

fig, ax = plt.subplots(figsize=(14, 8))
ax.plot(history, alpha=0.7, lw=1.5)
ax.set_title('Figure 3: Convergence of Beliefs in a DeGroot Model', fontsize=18)
ax.set_xlabel('Time Step', fontsize=14)
ax.set_ylabel('Belief', fontsize=14)
ax.axhline(history[-1, :].mean(), color='k', lw=2.5, linestyle='--', label='Final Consensus Belief')
ax.legend(fontsize=12)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

note("In a strongly connected network (where there is a path from any node to any other), beliefs are guaranteed to converge to a consensus. The speed of convergence and the final consensus value depend on the network structure, with the initial beliefs of the most influential (central) nodes carrying the most weight.")

<a id='diffusion'></a>
### Information Diffusion and Threshold Models
Another class of models focuses on the diffusion of behaviors or technologies that have network externalities, where an agent's benefit from adopting increases with the number of their neighbors who also adopt. The **Granovetter-style threshold model** is a classic example.

- Each agent has a **threshold** $q_i \in [0,1]$.
- An agent will adopt a new behavior if the fraction of their neighbors who have already adopted meets or exceeds their threshold.
- A small group of initial adopters can trigger a global cascade if they can convince their neighbors, who in turn convince their neighbors, and so on.

In [ ]:
sec("Simulating a Threshold Model of Diffusion")

class DiffusionThresholdModel:
    def __init__(self, G, initial_adopters):
        self.G = G
        self.adopters = set(initial_adopters)
        # Assign a random threshold to each node
        for node in G.nodes():
            G.nodes[node]['threshold'] = random.random()
        self.history = [len(self.adopters)]
        
    def step(self):
        newly_adopted = set()
        for node in self.G.nodes():
            if node in self.adopters: continue
            neighbors = list(self.G.neighbors(node))
            if not neighbors: continue
            
            adopter_neighbors = sum(1 for n in neighbors if n in self.adopters)
            if (adopter_neighbors / len(neighbors)) >= self.G.nodes[node]['threshold']:
                newly_adopted.add(node)
        
        self.adopters.update(newly_adopted)
        self.history.append(len(self.adopters))
        return len(newly_adopted)
    
    def run(self, max_steps=50):
        for _ in range(max_steps):
            if self.step() == 0: break
        return self.history

# --- Run and Plot ---
G_diff = nx.watts_strogatz_graph(n=500, k=8, p=0.1, seed=42)
initial_adopters = random.sample(list(G_diff.nodes()), 5)
diffusion_model = DiffusionThresholdModel(G_diff, initial_adopters)
adoption_history = diffusion_model.run()

fig, ax = plt.subplots(figsize=(14, 8))
ax.plot(adoption_history, lw=3, marker='o', markersize=5)
ax.set_title('Figure 4: S-Shaped Adoption Curve from a Threshold Model', fontsize=18)
ax.set_xlabel('Time Step'); ax.set_ylabel('Cumulative Number of Adopters')
ax.set_ylim(0, len(G_diff.nodes()))
ax.grid(True)
plt.show()

note("The model generates the classic 'S-shaped' adoption curve. The process starts slowly with a few innovators, then accelerates rapidly as cascades are triggered, and finally slows down as the market becomes saturated.")

<a id='financial-networks'></a>
## 3. Application 2: Financial Networks and Systemic Risk

<a id='contagion'></a>
### Modeling Contagion
We can use a network framework to model how the failure of one bank can cascade through an interconnected financial system, a phenomenon known as **systemic risk**. This was a key mechanism in the 2008 Global Financial Crisis. Here, we implement a simple model of contagion based on the work of Eisenberg and Noe (2001).

The simulation proceeds as follows:
- **Setup:** We create a directed network of banks. Each bank has a balance sheet with a capital buffer. A directed edge from Bank A to Bank B means A is a creditor of B (A has lent money to B).
- **Initial Shock:** We exogenously cause one bank to fail. Its capital is wiped out, and it defaults on its obligations to its creditors.
- **Contagion Mechanism:** When a bank fails, its creditors suffer a loss equal to the value of the loan. In each round, we check all solvent banks. A bank fails if the cumulative losses it incurs from its debtors failing exceed its capital buffer. This new failure can then trigger further losses for its own creditors in the next round.
- **Iteration:** We repeat this process until no new banks fail in a full round.

In [ ]:
sec("Animating Financial Contagion")
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def setup_financial_network(n_banks=100, m=3):
    G = nx.DiGraph(nx.barabasi_albert_graph(n=n_banks, m=m, seed=42))
    rng = np.random.default_rng(42)
    for node in G.nodes():
        G.nodes[node]['capital'] = 10.0
        G.nodes[node]['status'] = 'solvent'
    for u, v in G.edges():
        G.edges[u, v]['weight'] = rng.uniform(3, 7)
    return G

def animate_contagion(graph, initial_shock_nodes):
    G = graph.copy()
    pos = nx.spring_layout(G, seed=42)
    fig, ax = plt.subplots(figsize=(12, 10))
    plt.close(fig) # Prevent static plot
    
    history = []
    for node in initial_shock_nodes:
        if G.has_node(node): G.nodes[node]['status'] = 'initially_failed'
    history.append(G.copy())
    
    newly_failed_in_round = list(initial_shock_nodes)
    while newly_failed_in_round:
        failing_now = newly_failed_in_round
        newly_failed_in_round = []
        for failing_node in failing_now:
            for creditor, _ in G.in_edges(failing_node):
                if G.nodes[creditor]['status'] == 'solvent':
                    G.nodes[creditor]['capital'] -= G.edges[creditor, failing_node]['weight']
                    if G.nodes[creditor]['capital'] <= 0:
                        G.nodes[creditor]['status'] = 'failed_in_cascade'
                        newly_failed_in_round.append(creditor)
        if newly_failed_in_round:
            history.append(G.copy())

    def update(frame):
        ax.clear()
        G_t = history[frame]
        status_map = {'solvent': 'skyblue', 'initially_failed': 'orange', 'failed_in_cascade': 'red'}
        colors = [status_map[G_t.nodes[n]['status']] for n in G_t.nodes()]
        nx.draw(G_t, pos, ax=ax, node_color=colors, node_size=150, width=0.7, edge_color='gray', arrows=False)
        n_failed = sum(1 for n in G_t.nodes() if G_t.nodes[n]['status'] != 'solvent')
        ax.set_title(f'Financial Contagion - Round: {frame} | Total Failures: {n_failed}', fontsize=16)

    return FuncAnimation(fig, update, frames=len(history), interval=800)

# --- Run and Display ---
G_finance = setup_financial_network()
central_node = max(dict(G_finance.out_degree(weight='weight')).items(), key=lambda x: x[1])[0]
animation = animate_contagion(G_finance, [central_node])

note("Displaying animation of financial contagion. It may take a moment to render. The orange node is the initial failure, and red nodes fail in the subsequent cascade.")
display(HTML(animation.to_jshtml()))

<a id='production-networks'></a>
## 4. Application 3: Production Networks and Aggregate Fluctuations

<a id='acemoglu'></a>
### The Acemoglu et al. (2012) Framework
A major question in macroeconomics is whether microeconomic, firm-level shocks can cause aggregate business cycles. The traditional view was no; idiosyncratic shocks should average out in a large economy. However, **Acemoglu et al. (2012)** showed that in a world with input-output linkages (a production network), this is not true. Shocks to a few, highly central firms can propagate and create significant aggregate volatility.

We can model a simplified version of this. Imagine a multi-sector economy where each sector uses inputs from other sectors. The total output of sector $i$ is given by a Cobb-Douglas production function of its own labor and a composite of intermediate inputs from other sectors:
$$ Y_i = Z_i L_i^{1-\alpha} \prod_{j=1}^N Y_{ji}^{\alpha a_{ij}} $$ 
where $Z_i$ is productivity, $L_i$ is labor, and $Y_{ji}$ is the amount of goods from sector $j$ used by sector $i$. The coefficients $a_{ij}$ come from an input-output matrix. A negative shock to one sector's $Z_k$ will reduce its output, which then becomes a negative shock to all sectors that use goods from $k$ as an input, propagating through the economy.

In [ ]:
sec("Simulating a Shock in a Production Network")

class ProductionNetwork:
    def __init__(self, n_sectors=50):
        # Create a directed, random graph representing the input-output network
        self.G = nx.gnp_random_graph(n_sectors, p=0.2, directed=True, seed=42)
        self.io_matrix = nx.to_numpy_array(self.G)
        # Normalize rows to represent input shares
        self.io_matrix = self.io_matrix / self.io_matrix.sum(axis=1, keepdims=True, where=self.io_matrix.sum(axis=1, keepdims=True)>0)
        self.output = np.ones(n_sectors)
        self.productivity = np.ones(n_sectors)

    def run_shock_simulation(self, shock_node, shock_size=0.2, n_periods=10):
        history = []
        for t in range(n_periods):
            if t == 1:
                self.productivity[shock_node] *= (1 - shock_size)
            
            # Firms update output based on input availability (a simplification)
            input_availability = self.io_matrix @ self.output
            self.output = self.productivity * np.sqrt(input_availability) # Simple production function
            history.append(self.output.copy())
        return np.array(history)

# --- Run and Plot ---
prod_net = ProductionNetwork(n_sectors=50)
centrality = nx.degree_centrality(prod_net.G)
most_central_node = max(centrality, key=centrality.get)

shock_history = prod_net.run_shock_simulation(shock_node=most_central_node)
aggregate_gdp = shock_history.sum(axis=1)

fig, ax = plt.subplots(figsize=(14, 8))
ax.plot(aggregate_gdp / aggregate_gdp[0], lw=3, marker='o')
ax.set_title('Figure 6: Propagation of a Sector-Specific Shock in a Production Network', fontsize=18)
ax.set_xlabel('Time Since Shock')
ax.set_ylabel('Aggregate GDP (Indexed to 1)')
ax.axvline(1, color='r', linestyle='--', label='Productivity Shock to Central Sector')
ax.legend(); ax.grid(True)
plt.show()

note("A single shock to a central sector causes a significant and persistent drop in aggregate output. This demonstrates how network linkages can amplify microeconomic shocks into macroeconomic events. The density and structure of the production network are key determinants of an economy's resilience.")

<a id='summary'></a>
## 5. Summary and Key Takeaways

This notebook surveyed the application of network theory to economics, a field that provides powerful tools for understanding how the structure of connections shapes outcomes.
- **Network Topology Matters:** The structure of a network—whether it is random, small-world, or scale-free—is a crucial determinant of its behavior.
- **Centrality is Context-Dependent:** Different centrality measures (degree, betweenness, eigenvector) capture different notions of importance, and the relevant measure depends on the economic question being asked.
- **Social Networks** govern the diffusion of information, beliefs, and behaviors, with phenomena like consensus and S-shaped adoption curves emerging from the network structure.
- **Financial Networks** are a key channel for the propagation of shocks and systemic risk. The 'robust-yet-fragile' nature of scale-free networks makes them particularly vulnerable to the failure of central hubs.
- **Production Networks** can amplify idiosyncratic, firm-level shocks into aggregate business cycle fluctuations, providing a micro-founded theory of macroeconomic volatility.

<a id='exercises'></a>
## 6. Exercises

1.  **Network Robustness to Random Failure:** In the financial contagion example, we simulated a targeted attack on a central node. Modify the code to simulate a **random failure** by selecting 5 nodes at random to fail initially. Run this simulation on both the random and the scale-free networks. Which network topology is more robust to random shocks? Explain why this finding, combined with the result from the targeted attack, demonstrates the "robust-yet-fragile" property of scale-free networks.

2.  **Consensus in the DeGroot Model:** Does the DeGroot model always lead to a single consensus belief? Create a network with two separate, unconnected components (e.g., using `G1 = nx.erdos_renyi_graph(25, 0.3)` and `G2 = nx.erdos_renyi_graph(25, 0.3)`, then `G = nx.disjoint_union(G1, G2)`). Run the DeGroot simulation on this disconnected network. What happens to the long-run beliefs? How might this simple model relate to the concept of echo chambers or political polarization in social networks?

3.  **Strategic Network Formation:** In many economic contexts, the network structure itself is the result of strategic choices by agents. Research the foundational **Jackson-Wolinsky (1996)** model of strategic network formation. What is the key trade-off agents face when deciding whether to form a link? What is the difference between a socially efficient network and a pairwise stable network in their model, and why do they often not coincide?

4.  **Production Network Centrality:** In the production network model, the shock was applied to the node with the highest degree centrality. Would the aggregate impact be larger or smaller if the shock was instead applied to the node with the highest *betweenness* centrality? Modify the code to test this and explain the intuition for your result.

5.  **Information Design:** Imagine you are a social planner who wants to maximize the speed of information diffusion in the threshold model. You can choose the initial 5 adopters. Which nodes would you choose to maximize the final number of adopters after 10 steps? (Hint: Think about which centrality measure would identify the best 'seeds').